In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!ls "/content/drive/My Drive/Colab Notebooks"

dcn_mnist-1.py	   HW_1_Part_2.ipynb  __pycache__
HW_1_Part_1.ipynb  input_data.py      three_layer_neural_network.py


In [11]:
import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks')
print(sys.path)

['/content/drive/My Drive/Colab Notebooks', '/content/drive/My Drive/Colab Notebooks', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [12]:
from google.colab import files
src = list(files.upload().values())[0]
open('input_data.py','wb').write(src)
import input_data

Saving input_data.py to input_data (1).py


In [13]:
%load_ext tensorboard

In [18]:
import os
import time

import tensorflow as tf

if (tf.__version__.split('.')[0] == '2'):
    import tensorflow.compat.v1 as tf

    tf.disable_v2_behavior()

# Load MNIST dataset
#import input_data


import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

sess = tf.InteractiveSession()


def weight_variable(shape):
    '''
    Initialize weights
    :param shape: shape of weights, e.g. [w, h ,Cin, Cout] where
    w: width of the filters
    h: height of the filters
    Cin: the number of the channels of the filters
    Cout: the number of filters
    :return: a tensor variable for weights with initial values
    '''

    # IMPLEMENT YOUR WEIGHT_VARIABLE HERE
    initial = tf.truncated_normal(shape, stddev=0.1)
    W = tf.Variable(initial)
    return W


def bias_variable(shape):
    '''
    Initialize biases
    :param shape: shape of biases, e.g. [Cout] where
    Cout: the number of filters
    :return: a tensor variable for biases with initial values
    '''

    # IMPLEMENT YOUR BIAS_VARIABLE HERE
    initial = tf.constant(0.1, shape=shape)
    b = tf.Variable(initial)
    return b


def conv2d(x, W):
    '''
    Perform 2-D convolution
    :param x: input tensor of size [N, W, H, Cin] where
    N: the number of images
    W: width of images
    H: height of images
    Cin: the number of channels of images
    :param W: weight tensor [w, h, Cin, Cout]
    w: width of the filters
    h: height of the filters
    Cin: the number of the channels of the filters = the number of channels of images
    Cout: the number of filters
    :return: a tensor of features extracted by the filters, a.k.a. the results after convolution
    '''

    # IMPLEMENT YOUR CONV2D HERE
    h_conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

    return h_conv


def max_pool_2x2(x):
    '''
    Perform non-overlapping 2-D maxpooling on 2x2 regions in the input data
    :param x: input data
    :return: the results of maxpooling (max-marginalized + downsampling)
    '''

    # IMPLEMENT YOUR MAX_POOL_2X2 HERE
    h_max = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    return h_max


def stats_summary(var_name, var):
    ##monitor the statistics (min, max, mean, standard deviation, histogram)
    tf.summary.scalar("Min__" + var_name, tf.reduce_min(var))
    tf.summary.scalar("Max__" + var_name, tf.reduce_max(var))
    tf.summary.scalar("Mean__" + var_name, tf.reduce_mean(var))
    tf.summary.scalar("Std_Dev__" + var_name, tf.math.reduce_std(var))
    tf.summary.histogram("Histo__" + var_name, var)


def main():
    # Specify training parameters
    result_dir = './results/'  # directory where the results from the training are saved
    max_step = 5500  # the maximum iterations. After max_step iterations, the training will stop no matter what
    test_dir = result_dir + 'test/'
    val_dir = result_dir + 'val/'

    start_time = time.time()  # start timing

    # FILL IN THE CODE BELOW TO BUILD YOUR NETWORK

    # placeholders for input data and input labeles
    x = tf.placeholder(tf.float32, [None, 784], name='x')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y_')

    # Store Accuracies
    val_accuracy_ = tf.placeholder(tf.float32, shape=())
    test_accuracy_ = tf.placeholder(tf.float32, shape=())

    # reshape the input image
    x_image = tf.reshape(x, [-1, 28, 28, 1])

    # first convolutional layer
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    input_1 = conv2d(x_image, W_conv1) + b_conv1
    h_conv1 = tf.nn.relu(input_1)
    h_pool1 = max_pool_2x2(h_conv1)

    # second convolutional layer
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    input_2 = conv2d(h_pool1, W_conv2) + b_conv2
    h_conv2 = tf.nn.relu(input_2)
    h_pool2 = max_pool_2x2(h_conv2)

    # densely connected layer
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
    input_fc1 = tf.matmul(h_pool2_flat, W_fc1) + b_fc1
    h_fc1 = tf.nn.relu(input_fc1)

    # dropout
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    # softmax
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])
    y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2, name='y')

    # FILL IN THE FOLLOWING CODE TO SET UP THE TRAINING

    # setup training
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

    # Add a scalar summary for the snapshot loss.
    tf.summary.scalar(cross_entropy.op.name, cross_entropy)
    # Build the summary operation based on the TF collection of Summaries.
    # Layer-1
    stats_summary("W_conv1", W_conv1)
    stats_summary("b_conv1", b_conv1)
    stats_summary("input_1", input_1)
    stats_summary("h_conv1", h_conv1)
    stats_summary(" h_pool1", h_pool1)
    # Layer-2
    stats_summary("W_conv2", W_conv2)
    stats_summary("b_conv2", b_conv2)
    stats_summary("input_2", input_2)
    stats_summary("h_conv2", h_conv2)
    stats_summary("h_pool2", h_pool2)
    # densely connected layer
    stats_summary("W_fc1", W_fc1)
    stats_summary("b_fc1", b_fc1)
    stats_summary("h_pool2_flat", h_pool2_flat)
    stats_summary("input_fc1", input_fc1)
    stats_summary("h_fc1", h_fc1)
    # Output layer - Softmax
    stats_summary("W_fc2", W_fc2)
    stats_summary("b_fc2", b_fc2)
    stats_summary("y_conv ", y_conv)

    summary_op = tf.summary.merge_all()
    summary_op_test = tf.summary.scalar('test_accuracy', test_accuracy_)
    summary_op_val = tf.summary.scalar('val_accuracy', val_accuracy_)

    # Add the variable initializer Op.
    init = tf.initialize_all_variables()

    # Create a saver for writing training checkpoints.
    saver = tf.train.Saver()

    # Instantiate a SummaryWriter to output summaries and the Graph.
    summary_writer = tf.summary.FileWriter(result_dir, sess.graph)
    # Val test
    summary_writer_test = tf.summary.FileWriter(test_dir, sess.graph)
    summary_writer_val = tf.summary.FileWriter(val_dir, sess.graph)

    # Run the Op to initialize the variables.
    sess.run(init)

    # run the training
    for i in range(max_step):
        batch = mnist.train.next_batch(50)  # make the data batch, which is used in the training iteration.
        # the batch size is 50
        if i % 100 == 0:
            # output the training accuracy every 100 iterations
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0], y_: batch[1], keep_prob: 1.0})
            print("step %d, training accuracy %g" % (i, train_accuracy))

            # Update the events file which is used to monitor the training (in this case,
            # only the training loss is monitored)
            summary_str = sess.run(summary_op, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
            summary_writer.add_summary(summary_str, i)
            summary_writer.flush()

        # save the checkpoints every 1100 iterations
        if i % 1100 == 0 or i == max_step:
            checkpoint_file = os.path.join(result_dir, 'checkpoint')
            saver.save(sess, checkpoint_file, global_step=i)

            feed_dict_test = {x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}
            test_accuracy = accuracy.eval(feed_dict=feed_dict_test)
            print('step %d, test accuracy %g' % (i, test_accuracy))
            summary_str_test = sess.run(summary_op_test, feed_dict={test_accuracy_: test_accuracy})
            summary_writer_test.add_summary(summary_str_test, i)
            summary_writer_test.flush()

            feed_dict_val = {x: mnist.validation.images, y_: mnist.validation.labels, keep_prob: 1.0}
            val_accuracy = accuracy.eval(feed_dict=feed_dict_val)
            print('step %d, validation accuracy %g' % (i, val_accuracy))
            summary_str_val = sess.run(summary_op_val, feed_dict={val_accuracy_: val_accuracy})
            summary_writer_val.add_summary(summary_str_val, i)
            summary_writer_val.flush()

        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})  # run one train_step

    # print validation error
    print('validation accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.validation.images, y_: mnist.validation.labels, keep_prob: 1.0}))

    # print test error
    print("test accuracy %g" % accuracy.eval(feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

    stop_time = time.time()
    print('The training takes %f second to finish' % (stop_time - start_time))


if __name__ == "__main__":
    main()


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.16


InvalidArgumentError: ignored

In [ ]:
tensorboard --logdir=results --bind_all